In [1]:
import sys
import os
base_path = os.getcwd()
sys.path.append(os.path.join(base_path,'compressai'))

In [2]:
from compressai.zoo import mbt2018_mean
import torch
import torchvision
from torchvision import transforms
import torchvision.transforms as T
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split
y_hat =None

#задаём хук для того, чтобы достать параметры у_hat
def hook(module, input, output):
   global y_hat
   y_hat = output[0].clone()


#инициализация модели mbt2018_mean
model=mbt2018_mean(pretrained=True,quality=1) 
model.eval()
#установка хука для извлечения y_hat
handle = model.gaussian_conditional.register_forward_hook(hook)

#скачиваем датасет
transform = transforms.Compose([T.Resize((128,128)),
                                 T.ToTensor()])
full_dataset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
#выбираем его случайное подмножество
subset_indices = torch.randperm(len(full_dataset))[:2000]  
subset_dataset = torch.utils.data.Subset(full_dataset, subset_indices)

stl10_loader = DataLoader(subset_dataset, batch_size=1,
                                          shuffle=True, num_workers=1)

image,label=subset_dataset[0]
print(image.shape)

# im=T.ToPILImage()(image)
# im.show()
# x=torch.rand((1,3,128,128))
# o=model(x)
# print(y_hat.shape)


C:\Users\alber\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([1, 128, 128])


In [3]:
#создаём новый датасет из y_hat
y_hats = []
labels = []

for inputs, label in stl10_loader:
    with torch.no_grad(): 
        outputs = model(inputs.repeat(1,3,1,1))  
        y = y_hat  
        y_hats.append(y.cpu())
        labels.append(label.cpu())
    
y_hats = torch.cat(y_hats, dim=0)
labels = torch.cat(labels, dim=0)
torch.save(y_hats, 'y_hat_dataset/MNIST/train/y_hats.pt')
torch.save(labels, 'y_hat_dataset/MNIST/train/labels.pt')

